# Analyze the data file generated by moltemplate

## Conversion between RB-style dihedral function and Fourier (opls) style

In [1]:


def opls2RB(Flist):
    # convert the LAMMPS's "opls" dihedral form to RB form which is used in Gromacs
    # v_opls = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2 + f4*(1-cos(4fi))/2
    f0 = 0
    f1 = Flist[0]
    f2 = Flist[1]
    f3 = Flist[2]
    c0 = 0.5*(f1+2*f2+f3)+f0
    c1 = -0.5*(f1-3*f3)
    c2 = -f2
    c3 = -2*f3

    return [c0,c1,c2,c3] 


# # this function has been depreciated
# # because RB can only converted to the Fourier function with a constant term
# # see my dicussion in Onenote
# def Fourier2RB(Flist):
#     # lammps standard opls style dihedral:
#     # v_dihedral = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
# #     Flist = [f1,f2,f3,f4]
#     [f1,f2,f3,f4] = Flist
#     c0 = f2 + (f1+f3)/2.0
#     c1 = 0.5 * (-1.0*f1+3*f3)
#     c2 = -1.0*f2 + 4*f4
#     c3 = -2 * f3

#     return [c0,c1,c2,c3]



def Fourier2RB_Siu(Flist):
    # lopls paper by Siu et al. 2012
    # v_Fourier = f0 + f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    #
    [f0,f1,f2,f3] = Flist
    c0 = 0.5*(f1+2*f2+f3)+f0
    c1 = -0.5*(f1-3*f3)
    c2 = -f2
    c3 = -2*f3

    return [c0,c1,c2,c3]


# # this function has been depreciated
# # because RB can only converted to the Fourier function with a constant term
# # see my dicussion in Onenote
# def RB2Fourier(RBlist):
#     # convert RB dihedral form to lammps's opls standard form
#     # RB: v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
#     [c0,c1,c2,c3] = RBlist

#     f3 = -0.5*c3
#     f2 = -c2
#     f1 = -1.5*c3-2*c1

#     return [f1,f2,f3,0]


def RB2Fourier_Siu(RBlist):
    # convert RB dihedral form to Siu 2012's opls form
    # lopls paper by Siu et al. 2012
    # v_Fourier = f0 + f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    [c0,c1,c2,c3] = RBlist
    f3 = -0.5*c3
    f2 = -c2
    f1 = -1.5*c3-2*c1
    f0 = c0 - 0.5*(f1+2*f2+f3)
    
    return [f0,f1,f2,f3]

## Unit conversion

In [2]:
def kcal2kj (value):
    """
    convert unit from kcal/mol to kJ/mol 
    1 kJ/mol = 0.24 kcal/mol
    """

    return (float(value)*4.184)


def kj2kcal (value):
    """
    convert unit from kj/mol to kcal/mol 
    1 kJ/mol = 0.24 kcal/mol
    """
    
    return (float(value)/4.184)


def ev2kj (value):
    """
    convert unit from eV to kJ/mol 
    1 eV = 96 kJ/mol
    """
    
    return float(value)*96

## Read .data file and analyze
* The main goal is to extract the topology and the FF coefficent information
* Before execution, you had better comment the atom types in the "Mass" section of the data file. All information of bond, angle, dihedral, and improper rely on the correct naming of atom type.

In [3]:

from collections import defaultdict

with open("system.data") as file:
    data = file.readlines()

i_Mass = data.index("Masses\n")
i_Atoms = data.index("Atoms  # full\n")
i_Bonds = data.index("Bonds\n")
i_Angles = data.index("Angles\n")
i_Dihedrals = data.index("Dihedrals\n")
i_Impropers = data.index("Impropers\n")

massdata = data[i_Mass+2:i_Atoms-1]
atomdata = data[i_Atoms+2:i_Bonds-1]
bonddata = data[i_Bonds+2:i_Angles-1]
angleldata = data[i_Angles+2:i_Dihedrals-1]
dihedraldata = data[i_Dihedrals+2:i_Impropers-1]
improperdata = data[i_Impropers+2:]

atomtype = dict()
for line in massdata:
    num = line.split()[0]
    # toggle different naming styles, depending on what you need
#     ss = line.split()[5] # unique id for dihedral
#     name = line.split()[3] + "_" + ss[ss.index("d")+1:ss.index("i")-1]
    name = line.split()[3]+"("+line.split()[4]+')'
    name = line.split()[3]
    atomtype[num] = name

atoms = dict()
for line in atomdata:
    num = line.split()[0]
    atype = line.split()[2]
    atoms[num] = atomtype[atype]

#    bonds = dict()
bondtype = defaultdict(list)
for line in bonddata:
    b = line.split()
    if (atoms[b[2]], atoms[b[3]]) not in bondtype[b[1]] and (atoms[b[3]], atoms[b[2]]) not in bondtype[b[1]]:
        bondtype[b[1]].append((atoms[b[2]], atoms[b[3]]))

#    angles = dict()
angletype = defaultdict(list)
for line in angleldata:
    b = line.split()
    if (atoms[b[2]],atoms[b[3]],atoms[b[4]]) not in angletype[b[1]] and (atoms[b[4]],atoms[b[3]],atoms[b[2]]) not in angletype[b[1]]:
        angletype[b[1]].append((atoms[b[2]],atoms[b[3]],atoms[b[4]]))

#    dihedrals = dict()
dihedraltype = defaultdict(list)
for line in dihedraldata:
    b = line.split()
    c = (atoms[b[2]],atoms[b[3]],atoms[b[4]],atoms[b[5]])
    if c not in dihedraltype[b[1]] and c[::-1] not in dihedraltype[b[1]]:
        dihedraltype[b[1]].append(c)

impropertype = defaultdict(list)
for line in improperdata:
    if line == '\n':
        break
    b = line.split()
    c = (atoms[b[2]],atoms[b[3]],atoms[b[4]],atoms[b[5]])
#     if c not in impropertype[b[1]]:
    impropertype[b[1]].append(c)



**Extracting dihedral coefficients**

LAMMPS's OPLS form, kJ/mol

In [6]:
with open("system.in.settings") as file:
    data_coeff = file.readlines()

for line in data_coeff:
    b = line.split()
    if b[0] == "dihedral_coeff":
        d_para = [(kcal2kj(i)) for i in b[2:6]]

        print(b[1],f"{d_para[0]:7.5f} {d_para[1]:7.5f} {d_para[2]:7.5f} {d_para[3]:7.5f}",'--'.join(list(dihedraltype[b[1]][0])))

1 -0.00000 -0.00000 -0.00000 0.00000 O_2--C_2--CT--HC
2 -0.00000 0.00000 1.25520 0.00000 CT--CT--CT--HC
3 -0.00000 0.00000 1.25520 0.00000 HC--CT--CT--HC
4 0.00000 0.00000 0.82843 0.00000 C_2--OS--CT--HC
5 2.69742 -0.89680 0.74567 0.00000 CT--CT--CT--CT
6 13.05090 24.00658 -0.00000 0.00000 CT--OS--C_2--CT
7 -0.00000 24.00662 -0.00000 0.00000 CT--OS--C_2--O_2
8 -0.03105 0.00912 0.46781 0.00000 OS--C_2--CT--HC
9 -7.26091 -5.22347 2.61036 0.00000 CT--CT--OS--C_2
10 3.70276 -2.62295 -2.06413 0.00000 OS--C_2--CT--CT
11 -1.15483 5.17498 -2.80642 0.00000 CT--CT--C_2--O_2
12 -0.00891 0.07259 -1.23516 0.00000 C_2--CT--CT--HC
13 -9.65412 -2.62475 2.60224 0.00000 C_2--CT--CT--CT
14 9.45044 -4.28475 4.21371 0.00000 CT--CT--CT--OS
